In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from patchify import patchify
import PIL
from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000
import os
import shutil
import random
%matplotlib inline
import torchvision
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets, transforms
from torch import nn
from torch import optim
import torch.nn.functional as F
import cv2

In [ ]:
# Preprocessing map image
# Batch pre-processing Map image and save it into rotated_cropped_map_image file

# Currently the preprocessing only includes the rotated and cropped part.
# The image will store in rotated_cropped_map_image file

class Preprocessing(object):
    
    def __init__(self,path):
        self.path = path
        self.map_img = cv.imread(path)
        self.rotated_cropped_img = self.rotate_crop_map(self.map_img)
        self.saveImage(self.rotated_cropped_img)
        
    def rotate_crop_map(self,map_img):
        gray = cv.cvtColor(map_img, cv.COLOR_RGB2GRAY)
        gray = 255 - gray
        thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]

        # Compute rotated bounding box
        coords = np.column_stack(np.where(thresh > 0))
        center_rect, dims, angle = cv.minAreaRect(coords)

        if angle < -45:
            angle = -angle
        else:
            angle = 90-angle
        
        print("The rotated angle is %.3f" %angle)

        # Rotate image to deskew
        (h, w) = map_img.shape[:2]
        center = (w // 2, h // 2)
        M = cv.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv.warpAffine(map_img, M, (w, h), flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)

        gray = cv.cvtColor(rotated, cv.COLOR_BGR2GRAY)
        gray = 255*(gray < 128).astype(np.uint8) # To invert the text to white
        coords = cv.findNonZero(gray) # Find all non-zero points (text)
        x, y, w, h = cv.boundingRect(coords) # Find minimum spanning bounding box
        rect = rotated[y:y+h, x:x+w] 
        rect = cv.cvtColor(rect, cv.COLOR_BGR2RGB)

        return rect
    
    def saveImage(self,rotated_cropped_img):
        image_name = self.path.split("/")[3]
#         print(image_name)
        file_path = 'Bartho_img_collection/' + image_name
        cv.imwrite(file_path,cv.cvtColor(rotated_cropped_img, cv.COLOR_RGB2BGR))

# Batch pre-processing Map image and save it into rotated_cropped_map_image file
if __name__ == '__main__':
    map_name_list = os.listdir("E:\HP\Bartholomew_Edinburgh_Maps")
    for map_name in map_name_list:
        map_name_path = 'E:/HP/Bartholomew_Edinburgh_Maps/'+map_name
        print(map_name_path)
        
#         map_name_path = 'E:\HP\Bartholomew_Edinburgh_Maps\'
        
        Preprocessing(map_name_path)
        print("Rotated_cropped Success "+map_name)

E:/HP/Bartholomew_Edinburgh_Maps/103247211.27.tif
The rotated angle is 1.203
Rotated_cropped Success 103247211.27.tif
E:/HP/Bartholomew_Edinburgh_Maps/77775504.27.tif
The rotated angle is 0.000
Rotated_cropped Success 77775504.27.tif
E:/HP/Bartholomew_Edinburgh_Maps/82833386.27.tif
The rotated angle is 1.130


In [16]:
img = cv.imread('rotated_cropped_map_image\90719488.27.tif')
# "E:\HP\Code\rotated_cropped_map_image\90719488.27.tif"
# print(img.shape)"E:\HP\Code\rotated_cropped_map_image\90719485.27.tif"
cv.imwrite('90719488.27.jpg',img)

True

In [ ]:
"E:\HP\Code\rotated_cropped_map_image\82833386.27.tif"

In [9]:
# Based on rotated_cropped_map_image file, get the rotated_cropped_map_image
# Based on the patch_width, patchify the image into patches.
# Store the patches into file.
# Patches: Image into Patches(patchify into patches)
class Patches(object):
    
    def __init__(self,image_path,patch_width):
        self.image_path = image_path
        self.patch_width = patch_width
        self.tif_map_patches = None
        self.image2patches()
        self.getPatchShape()
    
    def image2patches(self):
        tif_map = PIL.Image.open(self.image_path)
        tif_map_np = np.array(tif_map)
        tif_map_patches = patchify(image = tif_map_np, patch_size = (self.patch_width, self.patch_width, 3),step = self.patch_width)
        print(tif_map_patches.shape)
        self.tif_map_patches = tif_map_patches.reshape(-1, self.patch_width, self.patch_width,3)
    
    def getPatchShape(self):
        print(f"Patches generated with shape: {self.tif_map_patches.shape}")
    
    def getPatches(self):
        return self.tif_map_patches
    
    def savePatchesImage(self,patches_file_path):
        for i in range(len(self.tif_map_patches)):
            patch = self.tif_map_patches[i]
            patch = Image.fromarray(patch)
            path = '{file_path}/patch_{num}.tif'.format(file_path=patches_file_path,num=i)
            patch.save(path)
            
if __name__ == '__main__':
    # Patches file.
    map_name_list = os.listdir('rotated_cropped_map_image')
    patch_width_list = [32,64,128,256]
    # Patch size you want to set up.
    for patch_width in patch_width_list:
        
#     patch_width = 32
    
        for map_name in map_name_list:
            # Generate the file that storing the patches.

            file = '{map_name}_patches_({size})'.format(map_name=map_name[:-4],size=patch_width)
            if os.path.exists(file):
                pass
            else:
                os.makedirs(file)

            map_name_path = 'rotated_cropped_map_image/'+map_name
            patch = Patches(map_name_path,patch_width)
            patch.savePatchesImage(file)

(493, 608, 1, 32, 32, 3)
Patches generated with shape: (299744, 32, 32, 3)
(443, 589, 1, 32, 32, 3)
Patches generated with shape: (260927, 32, 32, 3)
(318, 477, 1, 32, 32, 3)
Patches generated with shape: (151686, 32, 32, 3)
(317, 476, 1, 32, 32, 3)
Patches generated with shape: (150892, 32, 32, 3)


In [12]:
patch_width = 256
for map_name in map_name_list:
    # Generate the file that storing the patches.

    file = '{map_name}_patches_({size})'.format(map_name=map_name[:-4],size=patch_width)
    if os.path.exists(file):
        pass
    else:
        os.makedirs(file)

    map_name_path = 'rotated_cropped_map_image/'+map_name
    patch = Patches(map_name_path,patch_width)
    patch.savePatchesImage(file)

(61, 76, 1, 256, 256, 3)
Patches generated with shape: (4636, 256, 256, 3)
(55, 73, 1, 256, 256, 3)
Patches generated with shape: (4015, 256, 256, 3)
(39, 59, 1, 256, 256, 3)
Patches generated with shape: (2301, 256, 256, 3)
(39, 59, 1, 256, 256, 3)
Patches generated with shape: (2301, 256, 256, 3)
